In [5]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [7]:
subject = 'Pernambuco - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [8]:
data = pd.read_csv('2003_01_model_input_PE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - Consumo de Cimento (t)
0,2003,3.627100,0.688685,425.758313,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.294029,8.237381e+07,5.574893e+06,8.646979,7.571061e+07,1013.872000
1,2004,3.613756,0.692578,463.749993,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.221752,8.366033e+07,5.640541e+06,8.694752,7.623567e+07,951.037000
2,2005,3.503074,0.696127,471.505719,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.199101,8.484033e+07,5.649536e+06,8.733906,7.666319e+07,979.656000
3,2006,3.449433,0.699339,514.663879,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.107273,8.594487e+07,5.677153e+06,8.770881,7.705796e+07,1193.338000
4,2007,3.480239,0.702226,623.964785,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.081524,8.696379e+07,5.727693e+06,8.804784,7.741156e+07,1320.587000
5,2008,3.575368,0.704795,722.146507,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.049769,8.785657e+07,5.769035e+06,8.834432,7.768216e+07,1717.153000
6,2009,3.706614,0.707057,803.348817,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.078293,8.860216e+07,5.810149e+06,8.858245,7.790384e+07,1878.938000
7,2010,3.890411,0.709020,950.406487,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.042187,9.096526e+07,5.826469e+06,8.909890,7.856512e+07,2406.931000
8,2011,4.085239,0.710693,1079.328424,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,8.028882,9.301827e+07,5.792701e+06,8.937706,7.897343e+07,2496.113000
9,2012,4.276735,0.711600,1132.134532,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,12.192814,9.484266e+07,5.717659e+06,8.949260,7.926901e+07,2728.230000


In [9]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado
0,3.627100,0.688685,425.758313,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.294029,8.237381e+07,5.574893e+06,8.646979,7.571061e+07
1,3.613756,0.692578,463.749993,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.221752,8.366033e+07,5.640541e+06,8.694752,7.623567e+07
2,3.503074,0.696127,471.505719,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.199101,8.484033e+07,5.649536e+06,8.733906,7.666319e+07
3,3.449433,0.699339,514.663879,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.107273,8.594487e+07,5.677153e+06,8.770881,7.705796e+07
4,3.480239,0.702226,623.964785,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.081524,8.696379e+07,5.727693e+06,8.804784,7.741156e+07
5,3.575368,0.704795,722.146507,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.049769,8.785657e+07,5.769035e+06,8.834432,7.768216e+07
6,3.706614,0.707057,803.348817,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.078293,8.860216e+07,5.810149e+06,8.858245,7.790384e+07
7,3.890411,0.709020,950.406487,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.042187,9.096526e+07,5.826469e+06,8.909890,7.856512e+07
8,4.085239,0.710693,1079.328424,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,8.028882,9.301827e+07,5.792701e+06,8.937706,7.897343e+07
9,4.276735,0.711600,1132.134532,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,12.192814,9.484266e+07,5.717659e+06,8.949260,7.926901e+07


In [10]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      951.037000
1      979.656000
2     1193.338000
3     1320.587000
4     1717.153000
5     1878.938000
6     2406.931000
7     2496.113000
8     2728.230000
9     2982.021000
10    2983.632857
11    2632.837886
12    2308.455257
13    1813.941000
14    1645.004000
15    1690.846000
16    2018.834000
17    2022.411000
18    1940.927000
19            NaN
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [11]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado
0,-0.909391,-1.964470,-1.047122,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-0.867275,-1.686779,-0.132100,-2.296300,-2.161094
1,-0.941707,-1.590070,-0.902028,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-0.891581,-1.460188,0.258574,-1.741827,-1.695260
2,-1.209744,-1.248837,-0.872408,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-0.899198,-1.252358,0.312107,-1.287396,-1.315955
3,-1.339644,-0.939881,-0.707581,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.930080,-1.057817,0.476457,-0.858248,-0.965711
4,-1.265042,-0.662309,-0.290148,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.938739,-0.878357,0.777220,-0.464759,-0.651997
5,-1.034671,-0.415233,0.084820,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.949418,-0.721114,1.023251,-0.120648,-0.411915
6,-0.716834,-0.197759,0.394942,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.939826,-0.589796,1.267927,0.155728,-0.215239
7,-0.271738,-0.008998,0.956573,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.951968,-0.173588,1.365048,0.755145,0.371461
8,0.200072,0.151941,1.448942,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-0.956442,0.188003,1.164091,1.077983,0.733719
9,0.663815,0.239129,1.650615,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.443866,0.509330,0.717508,1.212088,0.995958


In [12]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      951.037000
1      979.656000
2     1193.338000
3     1320.587000
4     1717.153000
5     1878.938000
6     2406.931000
7     2496.113000
8     2728.230000
9     2982.021000
10    2983.632857
11    2632.837886
12    2308.455257
13    1813.941000
14    1645.004000
15    1690.846000
16    2018.834000
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [13]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado
17,2.140266,0.998695,-1.205634,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.070627,0.666787,-0.999485,-0.451599,-0.024935


In [14]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    2022.411
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [15]:
train_input.iloc[-(len(train_input)//5):]

,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado
14,0.955786,1.317253,-0.999691,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,1.246751,1.169858,-1.492793,0.419587,0.747715
15,1.424563,1.324346,-1.196005,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,1.162017,1.086335,-1.452034,0.155140,0.518015
16,1.800978,1.218437,-1.203936,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,1.147174,0.944355,-1.289950,-0.105390,0.300429


In [16]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [17]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [19]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2538133400, 324340144, 976321618, 140858469, 2813627537, 1110703108, 1079359953, 982763212, 284360456, 440320036, 4067727875, 3483136510, 1595330059, 4175415089, 2375364116, 3015807660, 984125088, 2130014580, 3292040611, 3641329582, 4093711557, 25393209, 229616632, 1229179504, 4137816082, 352902156, 1364057656, 2976925107, 979938959, 3929605442, 1278220423, 2846923522, 910586426, 2780074257, 550240050, 1949312665, 2901276643, 3212902687, 304363092, 745411585, 1862013598, 413050334, 3083184492, 3871222801, 2385153199, 2884017550, 4092926775, 2231302453, 3734153631, 337933447, 787850877, 417471051, 1479268497, 1822535531, 1424727596, 1333173160, 1776061464, 88712110, 4155854372, 1229604693, 316676300, 4050658978, 2615609218, 1982781813, 888775465, 3262590442, 1859619949, 2374236874, 1591822713, 3550847863, 762330158, 1886840309, 1382851479, 1028869772, 3735706335, 798474497, 4242181480, 3476924109, 4075297229, 3998108966, 3390671109, 660561965, 21870677, 889521647, 1198198788, 296949778

val_loss: 27714.056640625


Step: 88 ___________________________________________
val_loss: 62777.80078125


Step: 89 ___________________________________________
val_loss: 33625.84375


Step: 90 ___________________________________________
val_loss: 35038.796875


Step: 91 ___________________________________________
val_loss: 70020.5390625


Step: 92 ___________________________________________
val_loss: 31103.560546875


Step: 93 ___________________________________________
val_loss: 31685.375


Step: 94 ___________________________________________
val_loss: 27714.15625


Step: 95 ___________________________________________
val_loss: 19060.396484375


Step: 96 ___________________________________________
val_loss: 27714.193359375


Step: 97 ___________________________________________
val_loss: 27721.287109375


Step: 98 ___________________________________________
val_loss: 43196.46484375


Step: 99 ___________________________________________
val_loss: 27937.478515625


final_seed: 284360456

In [20]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 445ms/step - loss: 4350455.0000 - val_loss: 3051957.2500
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 4191668.7500 - val_loss: 915852.0000
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 7209935.0000 - val_loss: 1453812.6250
Epoch 4/10000
1/1 [==============================] - 0s 13ms/step - loss: 3374147.2500 - val_loss: 292575.8438
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 2869439.2500 - val_loss: 377265.7500
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 3602906.0000 - val_loss: 1858837.6250
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 3077176.5000 - val_loss: 7226031.5000
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 2685009.0000 - val_loss: 7003056.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 2832292.2500 - val_loss: 4729178

Epoch 74/10000
1/1 [==============================] - 0s 13ms/step - loss: 684040.3125 - val_loss: 1089386.1250
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 837357.6875 - val_loss: 720260.5625
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 792841.7500 - val_loss: 714156.5625
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 694462.5000 - val_loss: 703450.5000
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 729352.5000 - val_loss: 682023.1250
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 737294.0000 - val_loss: 648962.2500
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 599616.5625 - val_loss: 621872.0000
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 701926.0000 - val_loss: 549598.0625
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 673416.3125 - val_loss: 516164.3750


Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 142491.4062 - val_loss: 324695.9062
Epoch 222/10000
1/1 [==============================] - 0s 13ms/step - loss: 234398.7500 - val_loss: 182159.1875
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 204689.9062 - val_loss: 111799.6172
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 142071.5312 - val_loss: 89257.7266
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 192257.9219 - val_loss: 90368.0938
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 150168.9531 - val_loss: 97775.0312
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 138255.7656 - val_loss: 78001.8438
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 155071.6875 - val_loss: 80498.2891
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 161471.7969 - val_loss: 69158.898

1/1 [==============================] - 0s 14ms/step - loss: 73541.6719 - val_loss: 48131.4336
Epoch 369/10000
1/1 [==============================] - 0s 13ms/step - loss: 93125.0469 - val_loss: 56063.6602
Epoch 370/10000
1/1 [==============================] - 0s 14ms/step - loss: 101579.7969 - val_loss: 46859.6367
Epoch 371/10000
1/1 [==============================] - 0s 13ms/step - loss: 112736.7344 - val_loss: 63632.7656
Epoch 372/10000
1/1 [==============================] - 0s 14ms/step - loss: 85061.4922 - val_loss: 54967.2617
Epoch 373/10000
1/1 [==============================] - 0s 13ms/step - loss: 99000.1172 - val_loss: 63986.1602
Epoch 374/10000
1/1 [==============================] - 0s 14ms/step - loss: 80081.5859 - val_loss: 34423.6055
Epoch 375/10000
1/1 [==============================] - 0s 14ms/step - loss: 77513.9062 - val_loss: 52269.8867
Epoch 376/10000
1/1 [==============================] - 0s 14ms/step - loss: 89501.8281 - val_loss: 59979.0625
Epoch 377/10000
1/1 [===

1/1 [==============================] - 0s 13ms/step - loss: 95382.4688 - val_loss: 47476.2695
Epoch 517/10000
1/1 [==============================] - 0s 14ms/step - loss: 97286.0859 - val_loss: 49686.2617
Epoch 518/10000
1/1 [==============================] - 0s 21ms/step - loss: 71453.2188 - val_loss: 47301.8438
Epoch 519/10000
1/1 [==============================] - 0s 14ms/step - loss: 79726.5078 - val_loss: 48523.2773
Epoch 520/10000
1/1 [==============================] - 0s 13ms/step - loss: 69813.4609 - val_loss: 48806.4492
Epoch 521/10000
1/1 [==============================] - 0s 13ms/step - loss: 85972.5781 - val_loss: 50093.8008
Epoch 522/10000
1/1 [==============================] - 0s 13ms/step - loss: 85227.8359 - val_loss: 50914.7617
Epoch 523/10000
1/1 [==============================] - 0s 13ms/step - loss: 78156.8125 - val_loss: 47568.1758
Epoch 524/10000
1/1 [==============================] - 0s 13ms/step - loss: 42457.5977 - val_loss: 49024.4492
Epoch 525/10000
1/1 [=====

...

Epoch 1182/10000
1/1 [==============================] - 0s 13ms/step - loss: 89246.8281 - val_loss: 85270.0000
Epoch 1183/10000
1/1 [==============================] - 0s 14ms/step - loss: 128951.0312 - val_loss: 28062.0312
Epoch 1184/10000
1/1 [==============================] - 0s 14ms/step - loss: 98867.4219 - val_loss: 30818.0723
Epoch 1185/10000
1/1 [==============================] - 0s 13ms/step - loss: 98923.4922 - val_loss: 30923.0840
Epoch 1186/10000
1/1 [==============================] - 0s 13ms/step - loss: 80970.1406 - val_loss: 29084.0215
Epoch 1187/10000
1/1 [==============================] - 0s 13ms/step - loss: 126842.8203 - val_loss: 41802.4258
Epoch 1188/10000
1/1 [==============================] - 0s 13ms/step - loss: 93000.5000 - val_loss: 31821.2012
Epoch 1189/10000
1/1 [==============================] - 0s 13ms/step - loss: 103147.2422 - val_loss: 31706.2344
Epoch 1190/10000
1/1 [==============================] - 0s 13ms/step - loss: 76998.0156 - val_loss: 29425.185

Epoch 1329/10000
1/1 [==============================] - 0s 13ms/step - loss: 70694.5938 - val_loss: 39751.6953
Epoch 1330/10000
1/1 [==============================] - 0s 13ms/step - loss: 76168.3594 - val_loss: 28799.1953
Epoch 1331/10000
1/1 [==============================] - 0s 13ms/step - loss: 75191.9297 - val_loss: 42688.4375
Epoch 1332/10000
1/1 [==============================] - 0s 13ms/step - loss: 51464.8516 - val_loss: 30677.6113
Epoch 1333/10000
1/1 [==============================] - 0s 13ms/step - loss: 48122.3594 - val_loss: 32332.7578
Epoch 1334/10000
1/1 [==============================] - 0s 13ms/step - loss: 70609.2578 - val_loss: 27855.9590
Epoch 1335/10000
1/1 [==============================] - 0s 13ms/step - loss: 85347.1016 - val_loss: 28459.3379
Epoch 1336/10000
1/1 [==============================] - 0s 13ms/step - loss: 79997.1094 - val_loss: 39789.5195
Epoch 1337/10000
1/1 [==============================] - 0s 13ms/step - loss: 70012.0938 - val_loss: 36873.6758
E

Epoch 1403/10000
1/1 [==============================] - 0s 13ms/step - loss: 104535.0703 - val_loss: 33632.3242
Epoch 1404/10000
1/1 [==============================] - 0s 13ms/step - loss: 50161.9492 - val_loss: 33841.3945
Epoch 1405/10000
1/1 [==============================] - 0s 13ms/step - loss: 45782.1680 - val_loss: 33934.1797
Epoch 1406/10000
1/1 [==============================] - 0s 14ms/step - loss: 69553.8828 - val_loss: 41385.4336
Epoch 1407/10000
1/1 [==============================] - 0s 13ms/step - loss: 71425.9844 - val_loss: 27714.1094
Epoch 1408/10000
1/1 [==============================] - 0s 14ms/step - loss: 49321.6680 - val_loss: 33507.4492
Epoch 1409/10000
1/1 [==============================] - 0s 13ms/step - loss: 135618.0781 - val_loss: 27764.4004
Epoch 1410/10000
1/1 [==============================] - 0s 13ms/step - loss: 77800.9062 - val_loss: 42618.1055
Epoch 1411/10000
1/1 [==============================] - 0s 13ms/step - loss: 58812.2578 - val_loss: 31889.7969

Epoch 1477/10000
1/1 [==============================] - 0s 13ms/step - loss: 71414.9688 - val_loss: 34677.5820
Epoch 1478/10000
1/1 [==============================] - 0s 14ms/step - loss: 79368.2734 - val_loss: 36704.0430
Epoch 1479/10000
1/1 [==============================] - 0s 13ms/step - loss: 63264.7656 - val_loss: 41780.0039
Epoch 1480/10000
1/1 [==============================] - 0s 13ms/step - loss: 70098.3047 - val_loss: 39428.7852
Epoch 1481/10000
1/1 [==============================] - 0s 13ms/step - loss: 65526.2852 - val_loss: 33119.2852
Epoch 1482/10000
1/1 [==============================] - 0s 13ms/step - loss: 75252.3672 - val_loss: 47076.5117
Epoch 1483/10000
1/1 [==============================] - 0s 14ms/step - loss: 82438.3594 - val_loss: 32017.5527
Epoch 1484/10000
1/1 [==============================] - 0s 13ms/step - loss: 74538.4688 - val_loss: 42398.2734
Epoch 1485/10000
1/1 [==============================] - 0s 13ms/step - loss: 149730.3906 - val_loss: 36810.3711


Epoch 1551/10000
1/1 [==============================] - 0s 13ms/step - loss: 65575.9297 - val_loss: 42786.5312
Epoch 1552/10000
1/1 [==============================] - 0s 13ms/step - loss: 78755.0391 - val_loss: 42249.3672
Epoch 1553/10000
1/1 [==============================] - 0s 13ms/step - loss: 68398.0312 - val_loss: 37920.3047
Epoch 1554/10000
1/1 [==============================] - 0s 14ms/step - loss: 56657.6953 - val_loss: 35536.6914
Epoch 1555/10000
1/1 [==============================] - 0s 13ms/step - loss: 83061.6328 - val_loss: 27734.0156
Epoch 1556/10000
1/1 [==============================] - 0s 13ms/step - loss: 74484.5625 - val_loss: 30044.1777
Epoch 1557/10000
1/1 [==============================] - 0s 13ms/step - loss: 81352.0312 - val_loss: 27896.3809
Epoch 1558/10000
1/1 [==============================] - 0s 13ms/step - loss: 68375.0156 - val_loss: 31065.1934
Epoch 1559/10000
1/1 [==============================] - 0s 13ms/step - loss: 87259.2344 - val_loss: 27755.4121
E

In [21]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[2022.411](test_target) - [[1672.878]](prediction) = 349.53294824218756


In [22]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [23]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado
0,0.159485,-1.689052,-1.113145,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,1.850585,-1.543090,-1.619622,-1.644505,-1.656003
1,0.059807,-1.106763,-0.897581,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,1.013504,-1.050126,-0.829344,-1.061495,-1.053508
2,-0.766956,-0.576057,-0.853576,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,0.751177,-0.597974,-0.721056,-0.583677,-0.562927
3,-1.167633,-0.095549,-0.608698,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.312326,-0.174736,-0.388598,-0.132443,-0.109933
4,-0.937523,0.336147,0.011471,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.610536,0.215691,0.219803,0.281298,0.295815
5,-0.226941,0.720415,0.568550,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,-0.978316,0.557787,0.717490,0.643118,0.606329
6,0.753429,1.058643,1.029289,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,-0.647965,0.843479,1.212433,0.933718,0.860704
7,2.126333,1.352216,1.863689,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,-1.066123,1.748970,1.408895,1.563986,1.619522


train_target:


,Pernambuco - Consumo de Cimento (t)
0,951.037
1,979.656
2,1193.338
3,1320.587
4,1717.153
5,1878.938
6,2406.931
7,2496.113


test_input:


...

,Pernambuco - Valor Cimento Portland,Pernambuco - IDH,Pernambuco - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado
17,2.140266,0.998695,-1.205634,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.070627,0.666787,-0.999485,-0.451599,-0.024935


test_target:


,Pernambuco - Consumo de Cimento (t)
17,2022.411


1/1 [==============================] - 0s 34ms/step
Error: 349.53294824218756




[2359.48974609375,
 2517.419677734375,
 2716.209228515625,
 2917.1572265625,
 2618.320068359375,
 2470.691650390625,
 2217.649658203125,
 1962.45703125,
 2018.312255859375,
 1672.8780517578125]

In [24]:
display(targets)
display(predictions)

[2728.23,
 2982.021,
 2983.632857142857,
 2632.8378857142857,
 2308.455257142857,
 1813.941,
 1645.004,
 1690.846,
 2018.834,
 2022.411]

[2359.48974609375,
 2517.419677734375,
 2716.209228515625,
 2917.1572265625,
 2618.320068359375,
 2470.691650390625,
 2217.649658203125,
 1962.45703125,
 2018.312255859375,
 1672.8780517578125]

In [25]:
mae = mean_absolute_error(predictions, targets)
mae

354.60113890904023

In [26]:
porcentage = mae/np.mean(targets)
porcentage

0.1553482125611639